In [1]:
# standard setup
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cv2

# useful helper function to show images inline
from helpers import imshow
from helpers import colors

In [2]:
# Calculate contour centroids
def getContourCentroid(cntr):
    """Return the (cx,cy) contour center for cntr."""
    epsilon = 1e-5
    M = cv2.moments(cntr)
    cx = int(M['m10'] / (M['m00'] + epsilon))
    cy = int(M['m01'] / (M['m00'] + epsilon))
    return (cx,cy)

def isContourConvexQuadrilateral(cntr):
    """Test if a contour is a convex quadrilateral."""
    epsilon = 0.01*cv2.arcLength(cntr,True)
    approx = cv2.approxPolyDP(cntr,epsilon,True)
    return len(approx) == 4 and cv2.isContourConvex(approx)

# Capture a video

In [24]:
# setup capture
camera = cv2.VideoCapture(0)

height = 480
width = 640
fps = 30

camera.set(cv2.CAP_PROP_FRAME_WIDTH, width)
camera.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
camera.set(cv2.CAP_PROP_FPS, fps)
camera.set(cv2.CAP_PROP_AUTO_EXPOSURE, 0.1)

# define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi',fourcc, fps, (width, height))
record = False

# capture loop
while True:
    # get frame
    ret, frame = camera.read()

    # mirror the frame (my camera mirrors by default)
    frame = cv2.flip(frame, 1)

    # change color space to HSV
    gray = cv2.medianBlur(frame, 5)
    gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)
    
    # create binary mask
    _, mask = cv2.threshold(gray, 80, 255, cv2.THRESH_BINARY_INV)
            
    # DRAWING ------------------------------------------------------
    # Show frames
    cv2.imshow('gray', gray)
    cv2.imshow('mask', mask)
    # write the flipped frame
    record and out.write(frame)

    # KEYBOARD INPUT ----------------------------------------------
    k = cv2.waitKey(1)
    
    if k & 0xFF == ord('q'):
        break
    elif k & 0xFF == ord('p'):
        cv2.imwrite('frame.png', frame)
        cv2.imwrite('gray.png', gray)
        cv2.imwrite('mask.png', maskp)
        imshow(frame)
        imshow(gray)
    elif k & 0xFF == ord('r'):
        record = not record
        

# clean up
camera.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)  # extra waitKey sometimes needed to close camera window

-1

# Play button 1 video

In [25]:
import numpy as np
import cv2
from time import sleep

cap = cv2.VideoCapture('switch1.avi')
fps = 30

blob_area_thresh = [200, 1700]

while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret: 
        break
    
    # change color space to HSV
    gray = cv2.medianBlur(frame, 5)
    gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)
    
    # create binary mask
    _, mask = cv2.threshold(gray, 12, 255, cv2.THRESH_BINARY_INV)
            
    # Get contours of the blobs
    im2, contours, hierarchy = cv2.findContours(
        mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    # Calculate contour centroids and areas
    areas = [cv2.contourArea(c) for c in contours]
    centroids = [getContourCentroid(c) for c in contours]

    # Filter contours with small area
    contours = [c for c, a in zip(contours, areas) if a > blob_area_thresh[0]]
    centroids = [c for c, a in zip(centroids, areas) if a > blob_area_thresh[0]]
    areas = [a for a in areas if a > blob_area_thresh[0]]
    
    # Filter contours with large area
    contours = [c for c, a in zip(contours, areas) if a < blob_area_thresh[1]]
    centroids = [c for c, a in zip(centroids, areas) if a < blob_area_thresh[1]]
    areas = [a for a in areas if a < blob_area_thresh[1]]
    
    # Simplify contour shapes
    epsilons = [0.1 * cv2.arcLength(c, True) for c in contours]
    contours = [cv2.approxPolyDP(c, e, True) for c, e in zip(contours, epsilons)]
    
    # Get convex quadrilateral contours
    fil = [isContourConvexQuadrilateral(c) for c in contours]
    contours = [c for c, f in zip(contours, fil) if f]
    centroids = [c for c, f in zip(centroids, fil) if f]
    areas = [a for a, f in zip(areas, fil) if f]

    
    # DRAWING ------------------------------------------------------
    # Draw the contours on the frame image
    cv2.drawContours(frame, contours, -1, colors['red'], 3)

    # Draw the contour area text
    for c, a in zip(centroids, areas):
        cv2.putText(frame, str(a), c, cv2.FONT_HERSHEY_SIMPLEX,
                    1, colors['orange'], 1, cv2.LINE_AA)
    
    # Show frames
    cv2.imshow('frame', frame)
    cv2.imshow('gray', gray)
    cv2.imshow('mask', mask)
    
    k = cv2.waitKey(1)
    if k & 0xFF == ord('q'):
        break
    elif k & 0xFF == ord('p'):
        cv2.imwrite('keyimg.png', frame)

    sleep(1 / fps)
        
cap.release()
cv2.destroyAllWindows()

- Had to set auto-exposure off
- Setup is extremely light dependant. Shadows often break the system
- Created binary mask using gray image
- Got contours from mask
- Filtered contours with area in specified range. Camera view dependent.
- Filtered contours to only be convex quadrilaterals
- Can output off/on
- Can output press amount

In [5]:
import numpy as np
import cv2
from time import sleep

fps = 20 

class Switch(object):
    def __init__(self, cap, fps=None):
        self._cap = cap
        
    
    def _button_callback(self, contours, areas, centroids):
        pass
    
    
    def set_button_callback(self, callback):
        self._button_callback = callback
    
    
    def set_binary_thresh(self, lower, upper):
        self._lower_binary_thresh = lower
        self._upper_binary_thresh = upper
    
    
    def set_area_thresh(self, lower, upper):
        self._lower_area_thresh = lower
        self._upper_area_thresh = upper
    
    
    def _get_contours(self, mask):
        # Get contours of the blobs
        im2, contours, hierarchy = cv2.findContours(
            mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

        # Calculate contour centroids and areas
        areas = [cv2.contourArea(c) for c in contours]
        centroids = [getContourCentroid(c) for c in contours]

        # Filter contours with small area
        contours = [c for c, a in zip(contours, areas) if a > self._lower_area_thresh]
        centroids = [c for c, a in zip(centroids, areas) if a > self._lower_area_thresh]
        areas = [a for a in areas if a > self._lower_area_thresh]

        # Filter contours with large area
        contours = [c for c, a in zip(contours, areas) if a < self._upper_area_thresh]
        centroids = [c for c, a in zip(centroids, areas) if a < self._upper_area_thresh]
        areas = [a for a in areas if a < self._upper_area_thresh]

        # Simplify contour shapes
        epsilons = [0.06 * cv2.arcLength(c, True) for c in contours]
        contours = [cv2.approxPolyDP(c, e, True) for c, e in zip(contours, epsilons)]

        # Get convex quadrilateral contours
        fil = [isContourConvexQuadrilateral(c) for c in contours]
        contours = [c for c, f in zip(contours, fil) if f]
        centroids = [c for c, f in zip(centroids, fil) if f]
        areas = [a for a, f in zip(areas, fil) if f]
        
        # Set as class properties
        self.contours = contours
        self.contour_centroids = centroids
        self.contour_areas = areas
        
        
    def _draw_contours(self, frame):
        # Draw the contours on the frame image
        cv2.drawContours(frame, self.contours, -1, colors['red'], 3)

        # Draw the contour area text
        for c, a in zip(self.contour_centroids, self.contour_areas):
            cv2.putText(frame, str(a), c, cv2.FONT_HERSHEY_SIMPLEX,
                        1, colors['orange'], 1, cv2.LINE_AA)
    
    
    def play(self):
        while(self._cap.isOpened()):
            ret, frame = self._cap.read()
            if not ret: 
                break

            # change color space to HSV
            gray = cv2.medianBlur(frame, 5)
            gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)

            # create binary mask
            _, mask = cv2.threshold(
                gray, self._lower_binary_thresh, self._upper_binary_thresh, cv2.THRESH_BINARY_INV)
            
            # clean up mask
            kernel = np.ones((5,5),np.uint8)
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=1)
            
            # get the contours
            self._get_contours(mask)
            
            # draw the contours
            self._draw_contours(frame)
            
            # contour callback
            self._button_callback(self.contours, self.contour_areas, self.contour_centroids)

            # show frames
            cv2.imshow('frame', frame)
            cv2.imshow('gray', gray)
            cv2.imshow('mask', mask)

            # quit on q
            k = cv2.waitKey(1)
            if k & 0xFF == ord('q'):
                break
            elif k & 0xFF == ord('p'):
                cv2.imwrite('keyframe.png', frame)
                cv2.imwrite('keymask.png', mask)

            fps and sleep(1 / fps)

        self._cap.release()
        cv2.destroyAllWindows()
        

# Button 1 application demo

In [18]:
cap = cv2.VideoCapture('switch1.avi')

switch = Switch(cap, fps)
switch.set_area_thresh(200, 1700)
switch.set_binary_thresh(12, 255)

def callback(contours, areas, centroids):
    if len(areas):
        print(areas[0])

switch.set_button_callback(callback)
switch.play()

1179.5
344.5
337.0
337.5
334.5
1001.0
1027.0
940.0
670.5
1041.5
1250.5
225.5
1466.5
1520.0
1497.0
1498.5
1288.5
776.5
859.0


# Play button 2 video

In [7]:
cap = cv2.VideoCapture('switch2.avi')

switch = Switch(cap, fps)
switch.set_area_thresh(20, 300)
switch.set_binary_thresh(15, 255)
switch.play()

# Play button 3 video

In [8]:
cap = cv2.VideoCapture('switch3.avi')

switch = Switch(cap, fps)
switch.set_area_thresh(200, 1700)
switch.set_binary_thresh(30, 255)
switch.play()

# Play piano video

In [9]:
cap = cv2.VideoCapture('piano.avi')

switch = Switch(cap, fps)
switch.set_area_thresh(5, 150)
switch.set_binary_thresh(60, 255)
switch.play()

# Play slider video

In [7]:
cap = cv2.VideoCapture('slider.avi')

switch = Switch(cap, fps)
switch.set_area_thresh(20, 500)
switch.set_binary_thresh(70, 255)
switch.play()

# Play flip video

In [11]:
cap = cv2.VideoCapture('flip.avi')

switch = Switch(cap, fps)
switch.set_area_thresh(200, 1700)
switch.set_binary_thresh(12, 255)
switch.play()

# Play black key video

In [26]:
import numpy as np
import cv2
from time import sleep


class OtsuSwitch(object):
    def __init__(self, cap, fps=None):
        self._cap = cap
        
    
    def _button_callback(self, contours, areas, centroids):
        pass
    
    
    def set_button_callback(self, callback):
        self._button_callback = callback
    
    
    def set_area_thresh(self, lower, upper):
        self._lower_area_thresh = lower
        self._upper_area_thresh = upper
    
    
    def _get_contours(self, mask):
        # Get contours of the blobs
        im2, contours, hierarchy = cv2.findContours(
            mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

        # Calculate contour centroids and areas
        areas = [cv2.contourArea(c) for c in contours]
        centroids = [getContourCentroid(c) for c in contours]

        # Filter contours with small area
        contours = [c for c, a in zip(contours, areas) if a > self._lower_area_thresh]
        centroids = [c for c, a in zip(centroids, areas) if a > self._lower_area_thresh]
        areas = [a for a in areas if a > self._lower_area_thresh]

        # Filter contours with large area
        contours = [c for c, a in zip(contours, areas) if a < self._upper_area_thresh]
        centroids = [c for c, a in zip(centroids, areas) if a < self._upper_area_thresh]
        areas = [a for a in areas if a < self._upper_area_thresh]

        # Simplify contour shapes
        epsilons = [0.06 * cv2.arcLength(c, True) for c in contours]
        contours = [cv2.approxPolyDP(c, e, True) for c, e in zip(contours, epsilons)]

        # Get convex quadrilateral contours
        fil = [isContourConvexQuadrilateral(c) for c in contours]
        contours = [c for c, f in zip(contours, fil) if f]
        centroids = [c for c, f in zip(centroids, fil) if f]
        areas = [a for a, f in zip(areas, fil) if f]
        
        # Set as class properties
        self.contours = contours
        self.contour_centroids = centroids
        self.contour_areas = areas
        
        
    def _draw_contours(self, frame):
        # Draw the contours on the frame image
        cv2.drawContours(frame, self.contours, -1, colors['red'], 3)

        # Draw the contour area text
        for c, a in zip(self.contour_centroids, self.contour_areas):
            cv2.putText(frame, str(a), c, cv2.FONT_HERSHEY_SIMPLEX,
                        1, colors['orange'], 1, cv2.LINE_AA)
    
    
    def play(self):
        while(self._cap.isOpened()):
            ret, frame = self._cap.read()
            if not ret: 
                break

            # change color space to HSV
            gray = cv2.medianBlur(frame, 5)
            gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)

            # create binary mask with Otsu
            ret, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
            
            # clean up mask
            kernel = np.ones((5,5),np.uint8)
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=1)
            
            # get the contours
            self._get_contours(mask)
            
            # draw the contours
            self._draw_contours(frame)
            
            # contour callback
            self._button_callback(self.contours, self.contour_areas, self.contour_centroids)

            # show frames
            cv2.imshow('frame', frame)
            # cv2.imshow('gray', gray)
            cv2.imshow('mask', mask)

            # quit on q
            k = cv2.waitKey(1)
            if k & 0xFF == ord('q'):
                break
            elif k & 0xFF == ord('p'):
                cv2.imwrite('keyframe.png', frame)
                cv2.imwrite('keymask.png', mask)
                
            fps and sleep(1 / fps)

        self._cap.release()
        cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture('blackswitch.avi')
fps = 20
switch = OtsuSwitch(cap, fps)
switch.set_area_thresh(500, 5000)
switch.play()